# Detecção de Código de Barras em Produtos

**Grupo 8**
Processamento Digital de Imagens
Departamento de Computação - São Carlos, SP - Brasil

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import Image as IPyImage
import os
import glob

---
## Função Principal de Processamento
A função `detectar_codigo_barras` é o coração do nosso sistema. Ela recebe o caminho de uma imagem e uma série de parâmetros ajustáveis para realizar a detecção de códigos de barras. O processo envolve as seguintes etapas, detalhadas nos comentários do código a seguir.

In [ ]:
def detectar_codigo_barras(image_path, kernel_size=(25, 3), sobel_threshold=0, min_line_length=50):
    """
    Processa uma imagem para detectar a região de um código de barras.
    
    Args:
        image_path (str): Caminho para o arquivo de imagem.
        kernel_size (tuple): Tamanho do kernel para a operação de fechamento morfológico.
        sobel_threshold (int): Limiar para a binarização do gradiente Sobel. Se 0, usa THRESH_OTSU.
        min_line_length (int): Comprimento mínimo da linha para a detecção de Hough.
        
    Returns:
        tuple: Um dicionário com as imagens de cada etapa do processo e os pontos do retângulo (bbox) que envolve o código de barras detectado.
    """
    resultados = {}
    
    # --- 1. Leitura da Imagem ---
    # O que faz: Carrega a imagem do caminho especificado.
    # Por que: É o ponto de partida do processo. Inclui uma verificação para garantir que o arquivo existe.
    image = cv2.imread(image_path)
    if image is None:
        print(f"[ERRO] Imagem {image_path} não encontrada.")
        return None, None
    
    # --- 2. Redimensionamento ---
    # O que faz: Altera as dimensões da imagem para um tamanho padrão (600x400 pixels).
    # Por que: Garante que os parâmetros (como tamanho de kernel e de linha) funcionem de forma consistente 
    # em imagens de diferentes resoluções originais.
    image_resized = cv2.resize(image.copy(), (600, 400))
    resultados['original'] = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
    
    # --- 3. Conversão para Escala de Cinza ---
    # O que faz: Converte a imagem colorida (BGR) para escala de cinza.
    # Por que: A cor não é necessária para detectar a estrutura de um código de barras. A conversão 
    # simplifica a imagem para informações de luminância, reduzindo a complexidade computacional.
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    resultados['escala_cinza'] = gray
    
    # --- 4. Desfoque Gaussiano ---
    # O que faz: Aplica um filtro de desfoque (blur) para suavizar a imagem.
    # Por que: Reduz ruídos de alta frequência (como texturas finas ou imperfeições da imagem), 
    # o que ajuda a prevenir a detecção de falsas bordas na etapa seguinte.
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    resultados['desfoque_gaussiano'] = blurred
    
    # --- 5. Gradiente Sobel ---
    # O que faz: Calcula a derivada (gradiente) da imagem na direção X.
    # Por que: Códigos de barras são compostos por barras verticais. O operador Sobel no eixo X realça 
    # intensamente essas bordas verticais, fazendo com que a região do código de barras se destaque.
    gradX = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
    gradX = cv2.convertScaleAbs(gradX)
    resultados['gradiente_sobel'] = gradX
    
    # --- 6. Binarização (Limiarização) ---
    # O que faz: Converte a imagem de gradiente em uma imagem binária (preto e branco).
    # Por que: Separa as regiões de interesse (bordas fortes) do fundo. Usamos o método de Otsu 
    # (THRESH_OTSU) para encontrar um limiar ótimo automaticamente, tornando o método mais adaptável.
    if sobel_threshold == 0:
        _, thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    else:
        _, thresh = cv2.threshold(gradX, sobel_threshold, 255, cv2.THRESH_BINARY)
    resultados['binarizacao'] = thresh
    
    # --- 7. Fechamento Morfológico ---
    # O que faz: Aplica uma operação de fechamento (dilatação seguida de erosão) com um kernel retangular.
    # Por que: O kernel retangular e alongado (kernel_size) ajuda a fechar os espaços entre as barras 
    # verticais do código, unindo a região fragmentada em um bloco sólido.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    resultados['fechamento_morfologico'] = closed
    
    # --- 8. Refinamento Morfológico ---
    # O que faz: Aplica erosão e dilatação sequencialmente.
    # Por que: Remove pequenos ruídos brancos (pontos isolados) que possam ter sobrado após o fechamento, 
    # refinando a máscara da região candidata a ser um código de barras.
    closed = cv2.erode(closed, None, iterations=2)
    closed = cv2.dilate(closed, None, iterations=2)
    resultados['refinamento_morfologico'] = closed
        
    # --- 9. Unificação das Regiões ---
    # O que faz: Aplica mais operações morfológicas na máscara criada a partir das linhas de Hough.
    # Por que: As linhas detectadas podem estar separadas. Erosão e dilatação com um kernel grande unificam 
    # essas linhas em uma única região coesa que representa o código de barras completo.
    kernel_erode_dilate = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    mask_eroded = cv2.erode(closed, kernel_erode_dilate, iterations=1)
    mask_dilated = cv2.dilate(mask_eroded, kernel_erode_dilate, iterations=1)
    resultados['unificacao_regioes'] = mask_dilated
    
    # --- 10. Identificação e Contorno ---
    # O que faz: Encontra todos os contornos na máscara final e seleciona o de maior área.
    # Por que: Assumimos que o maior objeto coeso na imagem é o código de barras de interesse. 
    # Um retângulo de área mínima é então ajustado a este contorno para obter a localização precisa.
    contours, _ = cv2.findContours(mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    image_with_detection = image_resized.copy()
    bbox = None
    
    if contours:
        c = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(c)
        bbox = cv2.boxPoints(rect)
        bbox = np.intp(bbox)
        cv2.drawContours(image_with_detection, [bbox], -1, (0, 255, 0), 3)
    
    resultados['deteccao_final'] = cv2.cvtColor(image_with_detection, cv2.COLOR_BGR2RGB)
    
    return resultados, bbox

---
## Exemplos de Processamento por Etapas
Esta seção demonstra o fluxo de processamento da detecção de código de barras, mostrando o resultado de cada etapa. Você pode selecionar diferentes imagens e ajustar os parâmetros para observar o impacto nas transformações visuais. **Pelo menos três imagens de exemplo devem ser visualizadas aqui.**

In [ ]:
image_files = sorted(glob.glob('imagens/*.jpg'))

# Limitar a seleção a pelo menos 3 imagens para demonstração, se houver mais
display_image_files = image_files[:53] if len(image_files) >= 3 else image_files

param_kernel_width = widgets.IntSlider(value=25, min=5, max=50, description='Largura Kernel:')
param_kernel_height = widgets.IntSlider(value=3, min=1, max=10, description='Altura Kernel:')
param_threshold = widgets.IntSlider(value=0, min=0, max=255, description='Limiar Sobel:')
param_min_line = widgets.IntSlider(value=50, min=10, max=200, description='Min Linha:')
image_selector = widgets.Dropdown(options=display_image_files, description='Imagem:')

def update_processing(image_path, kernel_w, kernel_h, threshold, min_line):
    resultados, _ = detectar_codigo_barras(
        image_path, 
        kernel_size=(kernel_w, kernel_h),
        sobel_threshold=threshold,
        min_line_length=min_line
    )
    
    if resultados is None:
        return

    plt.figure(figsize=(20, 16))
    etapas = [
        ('original', 'Imagem Original'),
        ('escala_cinza', 'Escala de Cinza'),
        ('desfoque_gaussiano', 'Desfoque Gaussiano'),
        ('gradiente_sobel', 'Gradiente Sobel (X)'),
        ('binarizacao', 'Binarização'),
        ('fechamento_morfologico', 'Fechamento Morfológico'),
        ('refinamento_morfologico', 'Refinamento Morfológico'),
        ('unificacao_regioes', 'Regiões Unificadas'),
        ('deteccao_final', 'Detecção Final')
    ]
    
    for i, (key, title) in enumerate(etapas, 1):
        plt.subplot(4, 3, i)
        if key in ['original', 'deteccao_final']:
            plt.imshow(resultados[key])
        else:
            plt.imshow(resultados[key], cmap='gray')
        plt.title(title)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

widgets.interactive(
    update_processing,
    image_path=image_selector,
    kernel_w=param_kernel_width,
    kernel_h=param_kernel_height,
    threshold=param_threshold,
    min_line=param_min_line
)

---
## Validação Humana das Detecções
Nesta seção, você validará manualmente as detecções de código de barras para cada imagem. Para cada imagem, o sistema exibirá a detecção final e você deverá indicar se a detecção foi **bem-sucedida** ou **mal-sucedida**. Esta avaliação subjetiva é fundamental para entendermos a eficácia do algoritmo em cenários reais.

In [ ]:
human_validation_results = {
    'total_images': 0,
    'successful_detections': 0,
    'failed_detections': 0
}

output_area = widgets.Output()
success_button = widgets.Button(description="Sucesso", button_style='success', icon='check')
fail_button = widgets.Button(description="Falha", button_style='danger', icon='times')
start_validation_button = widgets.Button(description="Iniciar Validação", button_style='info')

# Esconde os botões de sucesso/falha inicialmente
buttons_box = widgets.HBox([success_button, fail_button])
buttons_box.layout.visibility = 'hidden'

# Preparar o iterador de imagens
all_image_files_for_validation = sorted(glob.glob('imagens/*.jpg'))
image_iterator = iter(all_image_files_for_validation)

def display_final_human_validation_stats():
    # Limpa a área para mostrar apenas os resultados finais
    with output_area:
        clear_output()
        print("Validação Humana Concluída!")

        # Verifica se alguma imagem foi validada para evitar divisão por zero
        if human_validation_results['total_images'] > 0:
            detection_rate = (human_validation_results['successful_detections'] / human_validation_results['total_images'] * 100)
            print("\n--- Resultados da Validação Humana ---")
            print(f"Total de imagens validadas: {human_validation_results['total_images']}")
            print(f"Detecções bem-sucedidas: {human_validation_results['successful_detections']}")
            print(f"Detecções mal-sucedidas: {human_validation_results['failed_detections']}")
            print(f"Taxa de Sucesso na Detecção: {detection_rate:.2f}%")
            
            # Gráfico de resultados
            plt.figure(figsize=(6, 4))
            validation_metrics = ['Bem-sucedidas', 'Mal-sucedidas']
            validation_values = [human_validation_results['successful_detections'], human_validation_results['failed_detections']]
            plt.bar(validation_metrics, validation_values, color=['lightgreen', 'salmon'])
            plt.title('Resultados da Validação Humana')
            plt.ylabel('Número de Imagens')
            for i, v in enumerate(validation_values):
                plt.text(i, v + 0.1, str(v), ha='center')
            plt.show()
        else:
            print("Nenhuma imagem foi validada.")

def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        
        # Bloco try...except para capturar o fim do iterador de forma segura
        try:
            # Tenta obter a próxima imagem
            img_path = next(image_iterator)
            
            resultados, _ = detectar_codigo_barras(img_path)
            if resultados is None:
                print(f"Não foi possível processar a imagem: {img_path}")
                on_button_clicked(None) # Pula para a próxima imagem automaticamente
                return

            plt.figure(figsize=(8, 6))
            plt.imshow(resultados['deteccao_final'])
            plt.title(f"Detecção para: {os.path.basename(img_path)}")
            plt.axis('off')
            plt.show()
            
            # Se for o primeiro clique, exibe os botões de validação
            if b is start_validation_button:
                 human_validation_results['total_images'] += 1
                 buttons_box.layout.visibility = 'visible'
                 start_validation_button.layout.visibility = 'hidden' # Esconde o botão de iniciar
            else: # para cliques subsequentes (sucesso/falha)
                 human_validation_results['total_images'] += 1


        except StopIteration:
            # O iterador acabou, finalizamos a validação
            buttons_box.layout.visibility = 'hidden' # Esconde os botões de sucesso/falha
            display_final_human_validation_stats()
            return

def on_success_click(b):
    human_validation_results['successful_detections'] += 1
    on_button_clicked(b) # Chama diretamente para processar a próxima imagem

def on_fail_click(b):
    human_validation_results['failed_detections'] += 1
    on_button_clicked(b) # Chama diretamente para processar a próxima imagem

# Associa as funções aos eventos de clique
start_validation_button.on_click(on_button_clicked)
success_button.on_click(on_success_click)
fail_button.on_click(on_fail_click)

# Exibe os widgets na tela
print("Clique em 'Iniciar Validação' para começar a avaliar as detecções.")
display(start_validation_button, output_area, buttons_box)

---
## Resultados e Conclusões
Esta seção apresenta os resultados finais de desempenho do sistema de detecção de código de barras, com base exclusivamente na **validação humana**. Discutimos a eficácia do algoritmo e as possíveis melhorias com base nos dados coletados manualmente.

In [ ]:
# Esta célula deve ser executada após a conclusão da "Validação Humana"
# Ela exibe um resumo dos resultados coletados para facilitar a visualização

print("\n=== RESULTADOS GERAIS DO ALGORITMO (Baseado na Validação Humana) ===")

if human_validation_results['total_images'] > 0:
    taxa_sucesso = (human_validation_results['successful_detections'] / human_validation_results['total_images'] * 100)
    
    print(f"Total de imagens validadas: {human_validation_results['total_images']}")
    print(f"Detecções bem-sucedidas: {human_validation_results['successful_detections']}")
    print(f"Detecções mal-sucedidas: {human_validation_results['failed_detections']}")
    print(f"Taxa de Sucesso (Humana): {taxa_sucesso:.2f}%")

    # Plot dos Resultados da Validação Humana
    plt.figure(figsize=(8, 6))
    human_metrics = ['Bem-sucedidas (Humana)', 'Mal-sucedidas (Humana)']
    human_values = [human_validation_results['successful_detections'], human_validation_results['failed_detections']]
    bars = plt.bar(human_metrics, human_values, color=['lightgreen', 'salmon'])
    plt.title('Resultados da Validação Humana')
    plt.ylabel('Número de Imagens')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, int(yval), va='bottom', ha='center') # va='bottom' para colocar o texto acima da barra
    plt.tight_layout()
    plt.show()
else:
    print("Nenhuma validação humana foi realizada ainda. Execute a seção de validação para gerar os resultados.")

print("\n--- Conclusões ---")

---
## Instruções para Execução

1. Crie uma pasta chamada `imagens` no mesmo diretório do notebook.
2. Coloque suas imagens de teste na pasta `imagens`.
3. **Certifique-se de que pelo menos 3 imagens existam na pasta `imagens` para que os exemplos funcionem adequadamente.**
4. Execute todas as células do notebook sequencialmente.
5. Na seção "Exemplos de Processamento por Etapas", use os controles interativos para ajustar parâmetros e visualizar as etapas de processamento.
6. Na seção "Validação Humana das Detecções", clique em "Iniciar Validação" e use os botões "Sucesso" ou "Falha" para classificar cada detecção exibida.
7. Os resultados da validação humana serão exibidos na seção "Resultados e Conclusões" após a finalização da etapa anterior.